In [1]:
import glob
from tqdm import tqdm
from subprocess import PIPE, Popen
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing
import os
import yaml

In [4]:
#os.chdir(f'../')

In [8]:
gnomad_vcf_location = "test"
tables_location = "test"
gnomad_version = "v4"

In [9]:
with open("gnomad_db/pkgdata/gnomad_columns.yaml") as f:
    columns = yaml.load(f, Loader=yaml.FullLoader)
columns = columns["base_columns"] + columns[gnomad_version]
print(len(columns))
columns[:10]

19


['CHROM',
 'POS',
 'REF',
 'ALT',
 'FILTER',
 'AC',
 'AN',
 'AF',
 'InbreedingCoeff',
 'MQ']

In [10]:
# get gnomAD files
files = glob.glob(f"{gnomad_vcf_location}/*.bgz")
print(len(files))
files

24


['/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr19.vcf.bgz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr1.vcf.bgz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chrY.vcf.bgz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr8.vcf.bgz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr21.vcf.bgz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr3.vcf.bgz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr14.vcf.bgz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr9.vcf.bgz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr16.vcf.bgz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4

In [11]:
# write gnomAD files to these tables:
tables_location = [f'{tables_location}/{file.split("/")[-1].replace(".vcf.bgz", "")}.tsv.gz' for file in files]
tables_location

['/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr19.tsv.gz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr1.tsv.gz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chrY.tsv.gz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr8.tsv.gz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr21.tsv.gz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr3.tsv.gz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr14.tsv.gz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr9.tsv.gz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.chr16.tsv.gz',
 '/mnt/biocluster/praktikum/tutorium_19/gnomad_db/data/exomes/gnomad.exomes.v4.0.sites.

In [12]:
cpu_count = int(multiprocessing.cpu_count())
cpu_count

12

In [13]:
# extract needed columns
# if running DIRECTLY from notebook, add module load i12g/bcftools; in the beginning of cmd
def create_table(file, table_location):
    query_string = "%" + "\t%".join(columns) + "\n"
    if not os.path.exists(table_location):
        cmd = f"bcftools query -f '{query_string}' {file} | gzip > {table_location}"
        p = Popen(cmd, shell=True, stdout=PIPE, stderr=PIPE)
        print(p.communicate())
    


In [14]:
# run bcftools in parallel
Parallel(cpu_count)(delayed(create_table)(file, table_location) for file, table_location in tqdm(zip(files, tables_location)))

24it [00:00, 61.20it/s] 


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: not found\n')
(b'', b'/bin/sh: 1: bcftools: no